<h1>Категории производства по пожарной и взрывопожарной опасности</h1>

In [12]:
import pandas as pd
import numpy as np
import pyodbc 

db_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-JSR6TV0G;'
                      'Database=Pro_t_londonSQL;'
                      'Trusted_Connection=yes;')

In [13]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Категории_пр', db_conn)
df.shape

(6, 3)

In [14]:
df

,Категория,Название,Текст_од
0,А,пожаровзрывоопасное,и обеспечивает пожаровзрывобезопасность при с...
1,Б,пожаровзрывоопасное,и обеспечивает пожаровзрывобезопасность при с...
2,В,пожароопасное,и обеспечивает пожаробезопасность при соблюде...
3,Г,безопасное,None
4,Д,безопасное,None
5,Е,взрывоопасное,и обеспечивает взрывобезопасность при соблюде...


In [15]:
df = df.replace("", np.nan)

<h2>Добавляем id</h2>

In [16]:
df.insert(0, 'id', range(1, len(df) + 1))

<h2>Меняем названия столбцов</h2>

In [17]:
df = df.rename(columns={"Категория": "category",
                        "Название": "name",
                        "Текст_од": "description"
                       })
df

,id,category,name,description
0,1,А,пожаровзрывоопасное,и обеспечивает пожаровзрывобезопасность при с...
1,2,Б,пожаровзрывоопасное,и обеспечивает пожаровзрывобезопасность при с...
2,3,В,пожароопасное,и обеспечивает пожаробезопасность при соблюде...
3,4,Г,безопасное,None
4,5,Д,безопасное,None
5,6,Е,взрывоопасное,и обеспечивает взрывобезопасность при соблюде...


<h2>Null для бд</h2>

In [18]:
df = df.where(pd.notnull(df), None)

In [19]:
df.head()

,id,category,name,description
0,1,А,пожаровзрывоопасное,и обеспечивает пожаровзрывобезопасность при с...
1,2,Б,пожаровзрывоопасное,и обеспечивает пожаровзрывобезопасность при с...
2,3,В,пожароопасное,и обеспечивает пожаробезопасность при соблюде...
3,4,Г,безопасное,None
4,5,Д,безопасное,None


<h1>Postgres</h1>

<h2>Создание таблицы</h2>

In [20]:
from psycopg2 import connect, sql, DatabaseError
import psycopg2.extras as extras

DocumentsKM

In [21]:
# Connect
try:
    conn = connect (
        dbname = "documentskm",
        user = "postgres",
        host = "localhost",
        password = "password"
    )
    cursor = conn.cursor()
except Exception as err:
    cursor = None
    print("Psycopg2 error:", err)
    
# Check if the connection was valid
if cursor != None:
    print("Connection successful")

Connection successful


In [22]:
cursor.execute(open("sql/5.sql", "r").read())
conn.commit()

<h2>Вставка данных</h2>

In [23]:
def execute_values(conn, df, table):
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [24]:
execute_values(conn, df, "fire_hazard_categories")

execute_values() done
